## **Ultrasound Nerve Segmentation** 

Importing necessary libraries 

In [ ]:
from fastai import *
from fastai.vision import *
import PIL

### Data

In [ ]:
mask = open_mask("/kaggle/input/ultrasound-nerve-segmentation/train/1_37_mask.tif", div=True)
mask.show()

In [ ]:
mask.data; mask.data.max()

In [ ]:
class SegmentationLabelListCustom(SegmentationLabelList):
    def open(self, fn): 
        return open_mask(fn, div=True)
class SegmentationItemListCustom(SegmentationItemList):
    _label_cls = SegmentationLabelListCustom

In [ ]:
label_func =  lambda fname: fname.with_name(str(fname.stem) + "_mask" + str(fname.suffix))
valid_func =  lambda fname: re.search("1_[0-9]*.tif$", str(fname)) or re.search("2_[0-9]*.tif$", str(fname))
train_filter = lambda fname: "mask" not in str(fname)
src = (SegmentationItemListCustom.from_folder('/kaggle/input/ultrasound-nerve-segmentation/train')
        .filter_by_func(train_filter)
        .split_by_valid_func(valid_func)
        .label_from_func(label_func, classes = ['0', '1']))

We will use the image resolution of 128 X 128 to speed up the training process.

For better accuracy - Use the full resolution provided in the original dataset.

In [ ]:
data = (src.transform(get_transforms(), size=128, tfm_y=True)
        .databunch(bs=4)
        .normalize(imagenet_stats))
data

In [ ]:
data.show_batch(2, figsize=(10, 7))

In [ ]:
metrics = [dice]

> ### Training 

In [ ]:
learn = unet_learner(data, models.resnet34, metrics=metrics)

In [ ]:
learn.model_dir = '/kaggle/working/'

In [ ]:
lr_find(learn)
learn.recorder.plot()

In [ ]:
lr=1e-4
learn.fit_one_cycle(5, lr)

In [ ]:
lr_find(learn)
learn.recorder.plot()

In [ ]:
learn.fit_one_cycle(5, slice(2e-5, lr/2))

In [ ]:
learn.show_results(rows=3, figsize=(9,11))

In [ ]:
learn.recorder.plot_losses()

In [ ]:
learn.unfreeze()

In [ ]:
lr_find(learn)
learn.recorder.plot()

In [ ]:
learn.fit_one_cycle(10, slice(1e-5, 1e-4))

In [ ]:
learn.recorder.plot_losses()

In [ ]:
lr_find(learn)
learn.recorder.plot()

In [ ]:
learn.fit_one_cycle(10, slice(3e-5, 6e-4))

### Results

In [ ]:
learn.show_results(rows=3, figsize=(9,11))

In [ ]:
learn.recorder.plot_metrics()

In [ ]:
learn.recorder.plot_losses()

In [ ]:
learn.save('/kaggle/working/stage-1')

In [ ]:
# data = (src.transform(get_transforms(), size=320, tfm_y=True)
#         .databunch(bs=4)
#         .normalize(imagenet_stats))
# data

In [ ]:
# learn_full_res = unet_learner(data, models.resnet34, metrics=metrics)

In [ ]:
# learn_full_res.load('/kaggle/working/stage-1')

In [ ]:
# learn_full_res.freeze()

In [ ]:
# learn_full_res.model_dir = '/kaggle/working/'

In [ ]:
# lr_find(learn_full_res)
# learn_full_res.recorder.plot()

In [ ]:
# learn_full_res.fit_one_cycle(5, 9e-5)

In [ ]:
# learn_full_res.unfreeze()

In [ ]:
# lr_find(learn_full_res)
# learn_full_res.recorder.plot()

In [ ]:
# learn_full_res.fit_one_cycle(10, slice(5e-6, 5e-5))